In [1]:
import psycopg2
import psycopg2.extras as extras

import time
import pandas as pd
import numpy as np

In [2]:
# access source RDS source database (datalake)
try:
    conn = psycopg2.connect("host=aoe21.cob0rzy4a66o.us-east-1.rds.amazonaws.com dbname=aoe21 user=tico password=1234567890")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

In [3]:
# establish cursor for source
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
conn.set_session(autocommit = True)

In [4]:
# access target RDS target database (datawarehouse)
try:
    conn_target = psycopg2.connect("host=aoe2warehouse.cm7nj2633ign.us-east-1.rds.amazonaws.com dbname=aoe2warehouse user=pasci password=123456789")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

In [5]:
# establish cursor for target
try:
    cur_target = conn_target.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
conn_target.set_session(autocommit = True)

In [6]:
postgreSQL_select_Query = "SELECT * FROM player_count"
cur.execute(postgreSQL_select_Query)

#Select all data from table player_count
player_count = cur.fetchall()
player_count = pd.DataFrame(player_count)

In [7]:
#Rename Columns
player_count.columns =['unix_time_stamp', 'player_count']

In [8]:
player_count.tail(10)

,unix_time_stamp,player_count
4324,1650795600,17014
4325,1650796200,17093
4326,1650796800,17152
4327,1650797400,17276
4328,1650798000,17420
4329,1650798600,17585
4330,1650799200,17785
4331,1650799800,17950
4332,1650800400,18208
4333,1650801000,18509


In [9]:
# create dublicate
player_count_adjusted = player_count

In [10]:
# unix_time_stamp to datetime
player_count_adjusted["datetime"] = pd.to_datetime(player_count_adjusted["unix_time_stamp"], unit = "s")
player_count_adjusted = player_count_adjusted[["datetime", "player_count"]]
player_count_adjusted

,datetime,player_count
0,2022-03-25 09:40:00,9424
1,2022-03-25 09:50:00,9596
2,2022-03-25 10:00:00,9771
3,2022-03-25 10:10:00,9845
4,2022-03-25 10:20:00,9973
...,...,...
4329,2022-04-24 11:10:00,17585
4330,2022-04-24 11:20:00,17785
4331,2022-04-24 11:30:00,17950
4332,2022-04-24 11:40:00,18208


In [11]:
# replace peaks with NaN (peaks smaller than 7000 counts)
player_count_adjusted.loc[player_count_adjusted.player_count < 7000, "player_count"] = np.nan
player_count_adjusted[player_count_adjusted.isna().any(axis = 1)]

C:\Users\pasci\AppData\Local\Temp\ipykernel_33020\3274706919.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_count_adjusted.loc[player_count_adjusted.player_count < 7000, "player_count"] = np.nan


,datetime,player_count
657,2022-03-29 23:10:00,NaN
1449,2022-04-04 11:10:00,NaN
1673,2022-04-06 00:30:00,NaN
2673,2022-04-12 23:10:00,NaN
3658,2022-04-19 19:20:00,NaN
3681,2022-04-19 23:10:00,NaN
3682,2022-04-19 23:20:00,NaN


In [12]:
# linear interpolation for matching the pattern
player_count_adjusted["player_count"] = player_count_adjusted["player_count"].interpolate()
player_count_adjusted[player_count_adjusted.isna().any(axis = 1)]

C:\Users\pasci\AppData\Local\Temp\ipykernel_33020\960951235.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_count_adjusted["player_count"] = player_count_adjusted["player_count"].interpolate()


,datetime,player_count


In [13]:
# convert player count to integer
player_count_adjusted["player_count"] = player_count_adjusted["player_count"].astype("int")
player_count_adjusted

C:\Users\pasci\AppData\Local\Temp\ipykernel_33020\2628660330.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_count_adjusted["player_count"] = player_count_adjusted["player_count"].astype("int")


,datetime,player_count
0,2022-03-25 09:40:00,9424
1,2022-03-25 09:50:00,9596
2,2022-03-25 10:00:00,9771
3,2022-03-25 10:10:00,9845
4,2022-03-25 10:20:00,9973
...,...,...
4329,2022-04-24 11:10:00,17585
4330,2022-04-24 11:20:00,17785
4331,2022-04-24 11:30:00,17950
4332,2022-04-24 11:40:00,18208


In [16]:
#cur_target.execute("DROP TABLE player_count;")
#cur_target.execute("DROP TABLE player_count_adjusted;")

In [17]:
#Create Table if it not exists
cur_target.execute("CREATE TABLE IF NOT EXISTS player_count_adjusted (datetime timestamp, player_count int);")

In [18]:
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()

In [19]:
execute_values(conn_target, player_count_adjusted, 'player_count_adjusted')

the dataframe is inserted
